<a href="https://colab.research.google.com/github/Huhan001/Advanced_Analytics_Thesis/blob/main/Advance_Analytic_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#We start by installing relevant packages that will aid with data reading.

import numpy as np
import pandas as pd

After the respective packages lets begin with importing the data.

In [ ]:
# Importing Data from Yahoo Finance.
!pip install fix-yahoo-finance==0.1.30
!pip install yfinance

import yfinance as yf
from datetime import datetime as dt

# Get the stock data for Apple
apple = yf.Ticker("AAPL")

# Get the historical market data
historical_data = apple.history(start='2015-06-01', end=dt.now())
# Print the data
historical_data

Now to cleaning the data and prepare it for Moving Average Analysis

In [ ]:
#Turned it to a DataFrama
apple_stock = pd.DataFrame(historical_data)

#Converted to datetime so as to maniputale the dates into timefame we can work with
apple_stock.index = pd.to_datetime(apple_stock.index).strftime('%Y-%m-%d')

#Excluded columns that we didn't need. Not neccesary for moving Average
apple_stock = apple_stock.drop(columns= ['Volume', 'Dividends', 'Stock Splits'])

#Rounded our data to 2 decimal places.
apple_stock =  apple_stock.round(2)

#For SME
apple_sme = apple_stock.round(2)


In [ ]:
# Visualization with the plotly package
!pip install plotly

#import the plotly package
import plotly.graph_objects as go


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=apple_stock.index,
                       open=apple_stock.Open, high=apple_stock.High,
                       low=apple_stock.Low, close=apple_stock.Close)])

fig.show()

Simple Moving Average

In [ ]:
#Before proceeding with operations, lets make sure that our Data is solid
apple_stock.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1923 entries, 2015-06-01 to 2023-01-18
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1923 non-null   float64
 1   High    1923 non-null   float64
 2   Low     1923 non-null   float64
 3   Close   1923 non-null   float64
dtypes: float64(4)
memory usage: 75.1+ KB


In [ ]:
#Choose [ Close ]
# The simple moving average over a period of 5 year
apple_stock['SMA_5'] = apple_stock.Close.rolling(5, min_periods=1).mean().round(2)

# The simple moving average over a period of 10 year
apple_stock['SMA_10'] = apple_stock.Close.rolling(10, min_periods=1).mean().round(2)

apple_stock.head(5)


,Open,High,Low,Close,SMA_5,SMA_10
Date,,,,,,
2015-06-01,29.56,29.81,29.50,29.62,29.62,29.62
2015-06-02,29.46,29.64,29.34,29.48,29.55,29.55
2015-06-03,29.64,29.71,29.47,29.52,29.54,29.54
2015-06-04,29.40,29.62,29.25,29.35,29.49,29.49
2015-06-05,29.38,29.42,29.12,29.19,29.43,29.43


In [ ]:
#Finally plotting our graphs with the SMA
previous_graph = go.Candlestick(x=apple_stock.index,
                       open=apple_stock.Open, high=apple_stock.High,
                       low=apple_stock.Low, close=apple_stock.Close)
#adding SMA
SMA5 = go.Scatter(x= apple_stock.index, y=apple_stock.SMA_5, mode='lines', name='SMA5')
SMA_10 = go.Scatter(x= apple_stock.index, y=apple_stock.SMA_10, mode='lines', name='SMA10')

#fit our graph
fig = go.Figure(data=[previous_graph])
fig.add_trace(SMA5) #Our SMA
fig.add_trace(SMA_10)

fig.show()

Compute the Mean Squared Error

In [ ]:
from sklearn.metrics import mean_squared_error
  
# Given values
Y_true = apple_stock.Close  # Y_true = Original value 
  
# calculated values
Y_pred = apple_stock.SMA_5  # Y_pred = calculated Simple Moving Average
  
# Calculation of Mean Squared Error (MSE)
mean_squared_error(Y_true,Y_pred)

3.525411388455539

In [ ]:
  
# SMA_10
Y_true = apple_stock.Close  # Y_true = Original value 
  
# calculated values
Y_pred = apple_stock.SMA_10  # Y_pred = calculated Simple Moving Average
  
# Calculation of Mean Squared Error (MSE)
mean_squared_error(Y_true,Y_pred)

8.06693822152886

Calculate Exponential Moving Average 

In [ ]:
# smoothing factor - 0.1
apple_sme['EMA_0.8'] = apple_sme.Close.ewm(alpha=0.8, adjust=False).mean().round(2)

# smoothing factor - 0.3
apple_sme['EMA_0.5'] = apple_sme.Close.ewm(alpha=0.5, adjust=False).mean().round(2)

apple_sme.head(5)

,Open,High,Low,Close,EMA_0.8,EMA_0.5
Date,,,,,,
2015-06-01,29.56,29.81,29.50,29.62,29.62,29.62
2015-06-02,29.46,29.64,29.34,29.48,29.51,29.55
2015-06-03,29.64,29.71,29.47,29.52,29.52,29.54
2015-06-04,29.40,29.62,29.25,29.35,29.38,29.44
2015-06-05,29.38,29.42,29.12,29.19,29.23,29.32


Calculate Mean Squared Error

In [ ]:
# EMA
Y_true= apple_sme.Close  # Y_true = Original value 
  
# calculated values
Y_pred = apple_sme['EMA_0.5']  # Y_pred = calculated Simple Moving Average
  
# Calculation of Mean Squared Error (MSE)
mean_squared_error(Y_true,Y_pred)

1.0099460218408736

In [ ]:
# EMA
Y_true= apple_sme.Close  # Y_true = Original value 
  
# calculated values
Y_pred = apple_sme['EMA_0.8']  # Y_pred = calculated Simple Moving Average
  
# Calculation of Mean Squared Error (MSE)
mean_squared_error(Y_true,Y_pred)

0.13273868954758183

Plotting 

In [ ]:
#Plotting with EMA
previous_graph = go.Candlestick(x=apple_sme.index,
                       open=apple_sme.Open, high=apple_sme.High,
                       low=apple_sme.Low, close=apple_sme.Close)
#adding SMA
EMA08 = go.Scatter(x= apple_sme.index, y=apple_sme['EMA_0.8'], mode='lines', name='EMA_8')
EMA05 = go.Scatter(x= apple_sme.index, y=apple_sme['EMA_0.5'], mode='lines', name='EMA_5')

#fit our graph
fig = go.Figure(data=[previous_graph])
fig.update_layout(yaxis_range=[115,180])

fig.add_trace(EMA08)

fig.show()

In [ ]:
previous_graph = go.Candlestick(x=apple_sme.index,
                       open=apple_sme.Open, high=apple_sme.High,
                       low=apple_sme.Low, close=apple_sme.Close)
#adding SMA
EMA08 = go.Scatter(x= apple_sme.index, y=apple_sme['EMA_0.8'], mode='lines', name='EMA_8')
EMA05 = go.Scatter(x= apple_sme.index, y=apple_sme['EMA_0.5'], mode='lines', name='EMA_5') #Added

#fit our graph
fig = go.Figure(data=[previous_graph])
fig.update_layout(yaxis_range=[115,180])

fig.add_trace(EMA08)
fig.add_trace(EMA05) #Added

fig.show()

Now for Google Data

In [ ]:
#Get Ticker 
google = yf.Ticker("GOOG")

#fetch data
google_hist= google.history(start='2015-06-01', end=dt.now())

#Turned it to a DataFrama
google_stock = pd.DataFrame(google_hist)

#Converted to datetime so as to maniputale the dates into timefame we can work with
google_stock.index = pd.to_datetime(google_stock.index).strftime('%Y-%m-%d')

#Excluded columns that we didn't need. Not neccesary for moving Average
google_stock = google_stock.drop(columns= ['Volume', 'Dividends', 'Stock Splits'])

#Rounded our data to 2 decimal places.
google_stock = google_stock.round(2)

#For SME
google_sme = google_stock.round(2)

#Before proceeding with operations, lets make sure that our Data is solid
google_sme.info()

# smoothing factor - 0.1
google_sme['EMA_0.8'] = google_sme.Close.ewm(alpha=0.8, adjust=False).mean().round(2)

# smoothing factor - 0.3
google_sme['EMA_0.3'] = google_sme.Close.ewm(alpha=0.3, adjust=False).mean().round(2)

google_sme.head(5)


<class 'pandas.core.frame.DataFrame'>
Index: 1926 entries, 2015-06-01 to 2023-01-23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1926 non-null   float64
 1   High    1926 non-null   float64
 2   Low     1926 non-null   float64
 3   Close   1926 non-null   float64
dtypes: float64(4)
memory usage: 75.2+ KB


,Open,High,Low,Close,EMA_0.8,EMA_0.3
Date,,,,,,
2015-06-01,26.84,26.84,26.49,26.70,26.70,26.70
2015-06-02,26.65,27.15,26.57,26.96,26.91,26.78
2015-06-03,27.00,27.17,26.86,27.02,27.00,26.85
2015-06-04,26.89,27.03,26.72,26.83,26.86,26.84
2015-06-05,26.82,26.86,26.63,26.67,26.71,26.79


In [ ]:
# EMA

from sklearn.metrics import mean_squared_error

Y_true= google_sme.Close  # Y_true = Original value 
  
# calculated values
Y_pred = google_sme['EMA_0.8']  # Y_pred = calculated Simple Moving Average
  
# Calculation of Mean Squared Error (MSE)
mean_squared_error(Y_true,Y_pred)

0.08673395638629273

In [ ]:

previous_graph = go.Candlestick(x=google_sme.index,
                       open=google_sme.Open, high=google_sme.High,
                       low=google_sme.Low, close=google_sme.Close)
#adding SMA
EMA08 = go.Scatter(x= google_sme.index, y=google_sme['EMA_0.8'], mode='lines', name='EMA_8')
EMA03 = go.Scatter(x= google_sme.index, y=google_sme['EMA_0.3'], mode='lines', name='EMA_3') #Added

#fit our graph
fig = go.Figure(data=[previous_graph])
fig.update_layout(yaxis_range=[50,160])

fig.add_trace(EMA08)
fig.add_trace(EMA03) #Added

fig.show()